In [1]:
import numpy as np
import pandas as pd
from scipy import interpolate


In [2]:
def filter_q1(df, split_col, filter_col="Score"):
    return df.groupby(split_col).apply(lambda x: x[x[filter_col] <= x[filter_col].quantile(0.25)]).reset_index(drop=True)
def filter_q3(df, split_col, filter_col="Score"):
    return df.groupby(split_col).apply(lambda x: x[x[filter_col] >= x[filter_col].quantile(0.75)]).reset_index(drop=True)


# Load data

In [3]:
#df_depth_rel_all = pd.read_csv("data/depth/rel_depth_statistics_combined.csv") 
#df_depth_abs_all = pd.read_csv("data/depth/met_depth_statistics_combined.csv") 

df_singleSVI_multiPar = pd.read_csv("data/labels/processed/singleSVI_multiPar_qscores.csv")
df_multiSVI_singlePar = pd.read_csv("data/labels/processed/multiSVI_singlePar_qscores.csv")

df_metadata = pd.read_csv("data/svi/metadata.csv")

In [4]:
# load each SVI depth calculations (in meters)
list_ids = df_metadata["uuid"]

depth_data = []

for uuid in list_ids:
    depth_val = np.load(f"data/depth/met_depth/{uuid}_depth.npy")
    depth_data.append({'uuid': uuid, 'depth': depth_val[0]})
df_depth_rel = pd.DataFrame(depth_data)
df_depth_rel['median'] = df_depth_rel['depth'].apply(np.median)
df_depth_rel # depth npy values for all svis that will be used

,uuid,depth,median
0,dc6eee81-9513-4edb-813b-05b83b5127be,"[30.194952, 30.022743, 29.850534, 29.795359, 2...",41.001743
1,50811b7a-5b0f-4581-92fa-017e6ff26ba0,"[22.900692, 22.54313, 22.185568, 21.76648, 21....",34.022415
2,35c8af9e-5e36-4810-9824-59b9b273fbcf,"[21.422682, 21.04729, 20.671898, 20.382578, 20...",31.951784
3,3a439403-305b-4be7-ba43-f8ff8e75c31c,"[21.728897, 21.46422, 21.199545, 20.82807, 20....",30.652710
4,3cc24150-c590-4d83-883c-44855010e338,"[28.704128, 28.510593, 28.317057, 28.211706, 2...",28.679663
...,...,...,...
395,416a4869-428a-475a-92bd-19f2611b2925,"[36.988827, 36.921658, 36.85449, 36.78854, 36....",30.508865
396,dad8c5a5-80cd-4d1e-a3cc-b429cdbb90c2,"[44.055176, 44.45355, 44.851925, 45.250237, 45...",30.887470
397,b7abb7f7-7031-4fc9-8f16-7e591dcdfbce,"[36.21824, 36.416855, 36.615475, 36.81323, 36....",41.819267
398,41a8b6ce-02df-47e8-b7d1-10cce55b6575,"[25.324125, 25.195616, 25.06711, 24.939291, 24...",23.466200


# Filtering only green answers

In [5]:
df_singleSVI_multiPar = df_singleSVI_multiPar[df_singleSVI_multiPar["Question"] == "green"]
df_multiSVI_singlePar = df_multiSVI_singlePar[df_multiSVI_singlePar["Question"] == "green"]


# Divide into Q1 and Q3 quantiles based on perception Q scores

In [6]:
# single SVI multi participant ratings
df_single_q1 = filter_q1(df_singleSVI_multiPar, "SVI_from")
df_single_q1['quantile'] = "Q1"
df_single_q3 = filter_q3(df_singleSVI_multiPar, "SVI_from")
df_single_q3['quantile'] = "Q3"
df_single_depth_q1q3 = pd.concat([df_single_q1, df_single_q3])

# multi SVI single participant ratings
df_multi_q1 = filter_q1(df_multiSVI_singlePar, "participants_from")
df_multi_q1['quantile'] = "Q1"
df_multi_q3 = filter_q3(df_multiSVI_singlePar, "participants_from")
df_multi_q3['quantile'] = "Q3"
df_multi_depth_q1q3 = pd.concat([df_multi_q1, df_multi_q3])

# location pair ratings (svi from and participants from)
df_par_from_svi_from = df_multiSVI_singlePar.merge(
    df_metadata[['Image number', 'city']].rename(columns={'city': 'SVI_from'}),
    left_on='Image',
    right_on='Image number',
    how='left'
)
df_par_from_svi_from = df_par_from_svi_from[df_par_from_svi_from['participants_from'] != 'All']
df_par_from_svi_from_q1 = filter_q1(df_par_from_svi_from, ["SVI_from", "participants_from"])
df_par_from_svi_from_q1['quantile'] = "Q1"
df_par_from_svi_from_q3 = filter_q3(df_par_from_svi_from, ["SVI_from", "participants_from"])
df_par_from_svi_from_q3['quantile'] = "Q3"
df_par_from_svi_from_q1q3 = pd.concat([df_par_from_svi_from_q1, df_par_from_svi_from_q3])

/var/folders/kw/9lnn_1r159n28p1krywmf8_w0000gq/T/ipykernel_37036/3541013002.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby(split_col).apply(lambda x: x[x[filter_col] <= x[filter_col].quantile(0.25)]).reset_index(drop=True)
/var/folders/kw/9lnn_1r159n28p1krywmf8_w0000gq/T/ipykernel_37036/3541013002.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby(split_col).apply(l

In [7]:
print(df_single_depth_q1q3.groupby(["SVI_from", "quantile"]).size()) # n >= 20
print(df_single_depth_q1q3.groupby(["SVI_from", "quantile"]).size().min())
print("\n")
print(df_multi_depth_q1q3.groupby(["participants_from", "quantile"]).size()) # n >= 30
print(df_multi_depth_q1q3.groupby(["participants_from", "quantile"]).size().min())
print("\n")
print(df_par_from_svi_from_q1q3.groupby(["SVI_from", "participants_from", "quantile"]).size()) # n >= 14
print(df_par_from_svi_from_q1q3.groupby(["SVI_from", "participants_from", "quantile"]).size().min())

SVI_from       quantile
Abuja          Q1           20
               Q3           20
All            Q1          100
               Q3          100
Amsterdam      Q1           20
               Q3           20
San Francisco  Q1           20
               Q3           20
Santiago       Q1           20
               Q3           20
Singapore      Q1           20
               Q3           20
dtype: int64
20


participants_from  quantile
All                Q1          100
                   Q3          100
Chile              Q1           76
                   Q3           77
Netherlands        Q1           77
                   Q3           77
Nigeria            Q1           80
                   Q3           77
Singapore          Q1           78
                   Q3           75
USA                Q1           75
                   Q3           75
dtype: int64
75


SVI_from       participants_from  quantile
Abuja          Chile              Q1          15
                            

# Divide into Q1 and Q3 quantiles based on depth median 

In [8]:
df_singleSVI_multiPar = df_singleSVI_multiPar.merge(df_depth_rel, on="uuid")
df_multiSVI_singlePar = df_multiSVI_singlePar.merge(df_depth_rel, on="uuid") 

In [9]:
# single SVI multi participant ratings
df_single_median_q1 = filter_q1(df_singleSVI_multiPar, "SVI_from", "median")
df_single_median_q1['quantile'] = "Q1"
df_single_median_q3 = filter_q3(df_singleSVI_multiPar, "SVI_from", "median")
df_single_median_q3['quantile'] = "Q3"
df_single_median_depth_q1q3 = pd.concat([df_single_median_q1, df_single_median_q3])

# multi SVI single participant ratings
df_multi_median_q1 = filter_q1(df_multiSVI_singlePar, "participants_from", "median")
df_multi_median_q1['quantile'] = "Q1"
df_multi_median_q3 = filter_q3(df_multiSVI_singlePar, "participants_from", "median")
df_multi_median_q3['quantile'] = "Q3"
df_multi_median_depth_q1q3 = pd.concat([df_multi_median_q1, df_multi_median_q3])

# location pair ratings (svi from and participants from)
df_par_from_svi_from_median = df_multiSVI_singlePar.merge(
    df_metadata[['Image number', 'city']].rename(columns={'city': 'SVI_from'}),
    left_on='Image',
    right_on='Image number',
    how='left'
)
df_par_from_svi_from_median = df_par_from_svi_from_median[df_par_from_svi_from_median['participants_from'] != 'All']
df_par_from_svi_from_median_q1 = filter_q1(df_par_from_svi_from_median, ["SVI_from", "participants_from"], "median")
df_par_from_svi_from_median_q1['quantile'] = "Q1"
df_par_from_svi_from_median_q3 = filter_q3(df_par_from_svi_from_median, ["SVI_from", "participants_from"], "median")
df_par_from_svi_from_median_q3['quantile'] = "Q3"
df_par_from_svi_from_median_q1q3 = pd.concat([df_par_from_svi_from_median_q1, df_par_from_svi_from_median_q3])

/var/folders/kw/9lnn_1r159n28p1krywmf8_w0000gq/T/ipykernel_37036/3541013002.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby(split_col).apply(lambda x: x[x[filter_col] <= x[filter_col].quantile(0.25)]).reset_index(drop=True)
/var/folders/kw/9lnn_1r159n28p1krywmf8_w0000gq/T/ipykernel_37036/3541013002.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby(split_col).apply(l

In [10]:
print(df_single_median_depth_q1q3.groupby(["SVI_from", "quantile"]).size()) # n >= 20
print(df_single_median_depth_q1q3.groupby(["SVI_from", "quantile"]).size().min())
print("\n")
print(df_multi_median_depth_q1q3.groupby(["participants_from", "quantile"]).size()) # n >= 30
print(df_multi_median_depth_q1q3.groupby(["participants_from", "quantile"]).size().min())
print("\n")
print(df_par_from_svi_from_median_q1q3.groupby(["SVI_from", "participants_from", "quantile"]).size()) # n >= 14
print(df_par_from_svi_from_median_q1q3.groupby(["SVI_from", "participants_from", "quantile"]).size().min())

SVI_from       quantile
Abuja          Q1           20
               Q3           20
All            Q1          100
               Q3          100
Amsterdam      Q1           20
               Q3           20
San Francisco  Q1           20
               Q3           20
Santiago       Q1           20
               Q3           20
Singapore      Q1           20
               Q3           20
dtype: int64
20


participants_from  quantile
All                Q1          100
                   Q3          100
Chile              Q1           76
                   Q3           76
Netherlands        Q1           77
                   Q3           77
Nigeria            Q1           75
                   Q3           75
Singapore          Q1           75
                   Q3           75
USA                Q1           75
                   Q3           75
dtype: int64
75


SVI_from       participants_from  quantile
Abuja          Chile              Q1          15
                            

In [11]:
# save csv file
df_single_median_depth_q1q3['grouping'] = "SVI_from"
df_multi_median_depth_q1q3['grouping'] = "participants_from"
df_par_from_svi_from_median_q1q3['grouping'] = "location_pairs"

df_median_q1q3 = pd.concat([df_single_median_depth_q1q3, df_multi_median_depth_q1q3, df_par_from_svi_from_median_q1q3])
df_median_q1q3.to_csv("data/depth/qscores_by_q1q3_median_depth.csv")
df_median_q1q3

,Image,Question,Score,Num_comparisons,uuid,city,Relabelled Name,Image number,green_view_index,SVI_from,depth,median,quantile,grouping,participants_from,Image number_x,Image number_y
0,119,green,6.517522,21,8e0aad6e-670c-4650-b4d3-6adc3c7da26c,Abuja,Image_119,119.0,0.274513,Abuja,"[22.074812, 21.85407, 21.633331, 21.414415, 21...",27.724718,Q1,SVI_from,NaN,NaN,NaN
1,154,green,4.585353,21,7e4701f6-fbe6-4c8a-b847-2468d6a4978d,Abuja,Image_154,154.0,0.350130,Abuja,"[15.1424465, 14.793273, 14.444099, 14.097632, ...",15.181726,Q1,SVI_from,NaN,NaN,NaN
2,141,green,5.599960,30,eee6dd59-4096-4c8c-ae6a-6dc872b7c187,Abuja,Image_141,141.0,0.250742,Abuja,"[29.630459, 29.431488, 29.23252, 29.034222, 28...",29.178806,Q1,SVI_from,NaN,NaN,NaN
3,103,green,3.848047,24,1fa2ab0a-14b4-4d2a-9522-7a8e5e6faa97,Abuja,Image_103,103.0,0.042903,Abuja,"[26.363457, 26.199791, 26.036125, 25.873568, 2...",30.333471,Q1,SVI_from,NaN,NaN,NaN
4,84,green,5.747354,23,c1e12031-ae65-4ad4-93e9-88a3c00cc96f,Abuja,Image_84,84.0,0.324266,Abuja,"[29.393862, 29.239325, 29.08479, 28.93081, 28....",29.220842,Q1,SVI_from,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,215,green,5.917108,6,a37b8477-4edf-4f49-b36f-2dd10e467f09,Singapore,Image_215,NaN,0.233810,Singapore,"[12.096749, 11.846966, 11.597183, 11.3474, 11....",39.576370,Q3,location_pairs,USA,215.0,215.0
381,231,green,4.500000,4,0a82c461-302e-45e9-afcb-5194bec3e123,Singapore,Image_231,NaN,0.233716,Singapore,"[27.620184, 27.597359, 27.574535, 27.552296, 2...",35.584648,Q3,location_pairs,USA,231.0,231.0
382,161,green,5.511905,8,a5e78fa1-a4b9-4062-a2f5-e4bd3b5854dd,Singapore,Image_161,NaN,0.236604,Singapore,"[22.736694, 22.546032, 22.35537, 22.16471, 22....",38.370567,Q3,location_pairs,USA,161.0,161.0
383,219,green,7.685185,6,5446831e-9c5b-4390-a583-6f0f3e3877ca,Singapore,Image_219,NaN,0.251703,Singapore,"[13.32514, 12.96369, 12.60224, 12.240788, 12.0...",35.485519,Q3,location_pairs,USA,219.0,219.0


# Calculate mean depths distribution within each group

In [12]:
def find_optimal_bins_individual(data):
    """Find optimal bins for individual distribution"""
    n = len(data)
    q75, q25 = np.percentile(data, [75, 25])
    iqr = q75 - q25
    
    if iqr == 0:
        return max(5, int(np.ceil(np.log2(n) + 1)))
    
    h = 2 * iqr / (n**(1/3))
    n_bins = int(np.ceil((data.max() - data.min()) / h))
    return max(5, min(n_bins, 100))

def compute_mean_histogram_individual_bins(df):
    """
    Use optimal bins for each distribution - more accurate per distribution
    """
    # Find individual optimal bins for each distribution
    individual_results = []
    
    for depth_array in df['depth']:
        # Find optimal bins for this specific distribution
        n_bins = find_optimal_bins_individual(depth_array)
        hist, bin_edges = np.histogram(depth_array, bins=n_bins, density=False)
        bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
        
        individual_results.append({
            'histogram': hist,
            'bin_centers': bin_centers,
            'bin_edges': bin_edges,
            'n_bins': n_bins
        })
    
    # To average these, we need to interpolate to common x-axis
    # Use the global range but with higher resolution
    all_values = np.concatenate(df['depth'].values)
    global_min, global_max = all_values.min(), all_values.max()
    
    # Create high-resolution common x-axis
    common_x = np.linspace(global_min, global_max, 1000)
    
    # Interpolate each histogram to common x-axis
    interpolated_hists = []
    for result in individual_results:
        f = interpolate.interp1d(
            result['bin_centers'], 
            result['histogram'], 
            kind='linear',
            bounds_error=False,  # Don't raise error for out-of-bounds
            fill_value=0.0       # Use 0 for extrapolation
        )
        interp_hist = f(common_x)
        interpolated_hists.append(interp_hist)
    
    # Average the interpolated histograms
    mean_histogram = np.mean(interpolated_hists, axis=0)
    
    return {
        'mean_histogram': mean_histogram,
        'bin_centers': common_x,
        'individual_results': individual_results,
        'approach': 'individual_bins'
    }


## single SVI multi participant

In [13]:
# compute mean hist for each combination of svi_from x quantile
single_list = []
for city in df_single_depth_q1q3['SVI_from'].unique():
    for quantile in df_single_depth_q1q3['quantile'].unique():
        df_aux = df_single_depth_q1q3[(df_single_depth_q1q3['SVI_from'] == city) & (df_single_depth_q1q3['quantile'] == quantile)]
        df_filtered = df_depth_rel[df_depth_rel['uuid'].isin(df_aux['uuid'])]
        mean_histogram_dict = compute_mean_histogram_individual_bins(df_filtered)
        single_list.append({'SVI_from': city,
                          'quantile': quantile,
                          'mean_depth_hist': mean_histogram_dict['mean_histogram']}) 
single_df = pd.DataFrame(single_list)
single_df['grouping'] = "SVI_from"
single_df

,SVI_from,quantile,mean_depth_hist,grouping
0,Abuja,Q1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",SVI_from
1,Abuja,Q3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",SVI_from
2,All,Q1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",SVI_from
3,All,Q3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",SVI_from
4,Amsterdam,Q1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",SVI_from
5,Amsterdam,Q3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",SVI_from
6,San Francisco,Q1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",SVI_from
7,San Francisco,Q3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",SVI_from
8,Santiago,Q1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",SVI_from
9,Santiago,Q3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",SVI_from


## multi SVI single participant

In [14]:
# compute mean hist for each combination of svi_from x quantile
multi_list = []
for city in df_multi_depth_q1q3['participants_from'].unique():
    for quantile in df_multi_depth_q1q3['quantile'].unique():
        df_aux = df_multi_depth_q1q3[(df_multi_depth_q1q3['participants_from'] == city) & (df_multi_depth_q1q3['quantile'] == quantile)]
        df_filtered = df_depth_rel[df_depth_rel['uuid'].isin(df_aux['uuid'])]
        mean_histogram_dict = compute_mean_histogram_individual_bins(df_filtered)
        multi_list.append({'participants_from': city,
                          'quantile': quantile,
                          'mean_depth_hist': mean_histogram_dict['mean_histogram']}) 
multi_df = pd.DataFrame(multi_list)
multi_df['grouping'] = "participants_from"
multi_df

,participants_from,quantile,mean_depth_hist,grouping
0,All,Q1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",participants_from
1,All,Q3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",participants_from
2,Chile,Q1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",participants_from
3,Chile,Q3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",participants_from
4,Netherlands,Q1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",participants_from
5,Netherlands,Q3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",participants_from
6,Nigeria,Q1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",participants_from
7,Nigeria,Q3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",participants_from
8,Singapore,Q1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",participants_from
9,Singapore,Q3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",participants_from


## location pairs

In [15]:
# compute mean hist for each combination of location pair x quantile
location_list = []

for city_participants in df_par_from_svi_from_q1q3['participants_from'].unique():
    for city_svi in df_par_from_svi_from_q1q3['SVI_from'].unique():
        for quantile in df_par_from_svi_from_q1q3['quantile'].unique():
            df_aux = df_par_from_svi_from_q1q3[(df_par_from_svi_from_q1q3['participants_from'] == city_participants) &
                                               (df_par_from_svi_from_q1q3['SVI_from'] == city_svi) & 
                                               (df_par_from_svi_from_q1q3['quantile'] == quantile)]
            df_filtered = df_depth_rel[df_depth_rel['uuid'].isin(df_aux['uuid'])]
            mean_histogram_dict = compute_mean_histogram_individual_bins(df_filtered)
            location_list.append({
                'participants_from': city_participants,
                'SVI_from': city_svi,
                'quantile': quantile,
                'mean_depth_hist': mean_histogram_dict['mean_histogram']}) 
location_df = pd.DataFrame(location_list)
location_df['grouping'] = "location_pairs"
location_df

,participants_from,SVI_from,quantile,mean_depth_hist,grouping
0,Chile,Abuja,Q1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.5333333333333333, ...",location_pairs
1,Chile,Abuja,Q3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",location_pairs
2,Chile,Amsterdam,Q1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",location_pairs
3,Chile,Amsterdam,Q3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",location_pairs
4,Chile,San Francisco,Q1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",location_pairs
5,Chile,San Francisco,Q3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",location_pairs
6,Chile,Santiago,Q1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",location_pairs
7,Chile,Santiago,Q3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",location_pairs
8,Chile,Singapore,Q1,"[0.0, 0.0, 0.0, 3.44913690657322, 2.8607635801...",location_pairs
9,Chile,Singapore,Q3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",location_pairs


## Combine results and save to csv

In [16]:
results_df = pd.concat([single_df, multi_df, location_df])
results_df['mean_depth_hist'] = results_df['mean_depth_hist'].apply(lambda x: x.tolist() if isinstance(x, np.ndarray) else x)
results_df.to_csv("data/depth/mean_depth_hist_by_q1q3_qscores.csv")
results_df

,SVI_from,quantile,mean_depth_hist,grouping,participants_from
0,Abuja,Q1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",SVI_from,NaN
1,Abuja,Q3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",SVI_from,NaN
2,All,Q1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",SVI_from,NaN
3,All,Q3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",SVI_from,NaN
4,Amsterdam,Q1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",SVI_from,NaN
...,...,...,...,...,...
45,San Francisco,Q3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",location_pairs,USA
46,Santiago,Q1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",location_pairs,USA
47,Santiago,Q3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",location_pairs,USA
48,Singapore,Q1,"[0.0, 0.0, 0.0, 2.840465687766181, 2.355922948...",location_pairs,USA
